In [1]:
import os
from dotenv import load_dotenv
from dotenv import dotenv_values
import json

In [2]:
OPENAI_API_VERSION ='2023-07-01-preview'
ENV_FILE_PATH = "../.env"
load_dotenv(ENV_FILE_PATH)
values_env = dotenv_values(ENV_FILE_PATH)
os.environ["AZURE_OPENAI_API_KEY"] =values_env['AZURE_OPENAI_KEY']
os.environ["AZURE_OPENAI_ENDPOINT"] =values_env['AZURE_OPENAI_ENDPOINT']
os.environ["OPENAI_API_VERSION"] =OPENAI_API_VERSION
deployment = values_env['AZURE_OPENAI_DEPLOYMENT_ID_AGENTS']

In [3]:
from langchain.chat_models import AzureChatOpenAI

In [4]:
from langchain.tools import tool
from langchain.agents import AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [5]:
llm_chat = AzureChatOpenAI(azure_deployment=deployment,temperature=0)

c:\Ambarish\AZURE-AI-VECTOR-SEARCH\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.AzureChatOpenAI instead.
  warn_deprecated(
c:\Ambarish\AZURE-AI-VECTOR-SEARCH\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [6]:
@tool
def add_two_numbers(num1, num2):
    """Function that adds 2 numbers."""""
    return json.dumps({"result": num1 + num2})

@tool
def multiply_two_numbers(num1, num2):
    """Function that multiplies 2 numbers."""""
    return json.dumps({"result": num1 * num2})

In [7]:
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

In [8]:
db = SQLDatabase.from_uri("sqlite:///../EUROSOCCER/database.sqlite")  

In [9]:
db

In [10]:
@tool
def query_european_soccer(query):
    """Function that helps with Country, League, 
    Match, Player, Player_Attributes, Team, Team_Attributes
    of European soccer."""""
    agent_executor = create_sql_agent(
    llm=llm_chat,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm_chat),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)
    response = agent_executor.run(query)
    return json.dumps({"result": response})


In [11]:
tools = [add_two_numbers, multiply_two_numbers, query_european_soccer]

In [12]:
prompt = ChatPromptTemplate.from_messages(
[
    ("system","You are very powerful assistant that helps\
                users perform tasks in the terminal."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

llm_with_tools = llm_chat.bind(functions=[format_tool_to_openai_function(t) for t in tools])


In [13]:
agent = (
{
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_function_messages(
        x["intermediate_steps"])
}
| prompt
| llm_with_tools
| OpenAIFunctionsAgentOutputParser())

In [14]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [15]:
action_input = "Get the count of players and add 100"

In [16]:
response = agent_executor.invoke({"input": action_input})



> Entering new AgentExecutor chain...

Invoking: `query_european_soccer` with `{'query': 'SELECT COUNT(*) FROM Player'}`




> Entering new AgentExecutor chain...


c:\Ambarish\AZURE-AI-VECTOR-SEARCH\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The class `ZeroShotAgent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use create_react_agent instead.
  warn_deprecated(
c:\Ambarish\AZURE-AI-VECTOR-SEARCH\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The class `Agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
c:\Ambarish\AZURE-AI-VECTOR-SEARCH\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
c:\Ambarish\AZURE-AI-VECTOR-SEARCH\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning

Action: sql_db_list_tables
Action Input: 
Observation: Country, League, Match, Player, Player_Attributes, Team, Team_Attributes
Thought:

c:\Ambarish\AZURE-AI-VECTOR-SEARCH\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


I can query the Player table to get the count of all the players.
Action: sql_db_schema
Action Input: Player
Observation: 
CREATE TABLE "Player" (
	id INTEGER, 
	player_api_id INTEGER, 
	player_name TEXT, 
	player_fifa_api_id INTEGER, 
	birthday TEXT, 
	height INTEGER, 
	weight INTEGER, 
	PRIMARY KEY (id)
)

/*
3 rows from Player table:
id	player_api_id	player_name	player_fifa_api_id	birthday	height	weight
1	505942	Aaron Appindangoye	218353	1992-02-29 00:00:00	182.88	187
2	155782	Aaron Cresswell	189615	1989-12-15 00:00:00	170.18	146
3	162549	Aaron Doran	186170	1991-05-13 00:00:00	170.18	163
*/
Thought:

c:\Ambarish\AZURE-AI-VECTOR-SEARCH\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


I can now construct the query to get the count of all the players.
Action: sql_db_query
Action Input: SELECT COUNT(*) FROM Player
Observation: [(11060,)]
Thought:

c:\Ambarish\AZURE-AI-VECTOR-SEARCH\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


I now know the final answer.
Final Answer: There are 11,060 players in the database.

> Finished chain.
{"result": "There are 11,060 players in the database."}
Invoking: `add_two_numbers` with `{'num1': 11060, 'num2': 100}`


{"result": 11160}The count of players is 11,060 and if you add 100 to it, the result is 11,160.

> Finished chain.


In [17]:
response["output"]

'The count of players is 11,060 and if you add 100 to it, the result is 11,160.'

In [18]:
action_input = "Get the average height of players in European Soccer \
and add 100."

response = agent_executor.invoke({"input": action_input})



> Entering new AgentExecutor chain...

Invoking: `query_european_soccer` with `{'query': 'SELECT AVG(height) FROM Player'}`




> Entering new AgentExecutor chain...


c:\Ambarish\AZURE-AI-VECTOR-SEARCH\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The class `ZeroShotAgent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use create_react_agent instead.
  warn_deprecated(
c:\Ambarish\AZURE-AI-VECTOR-SEARCH\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The class `Agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(
c:\Ambarish\AZURE-AI-VECTOR-SEARCH\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
c:\Ambarish\AZURE-AI-VECTOR-SEARCH\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning

Action: sql_db_list_tables
Action Input: 
Observation: Country, League, Match, Player, Player_Attributes, Team, Team_Attributes
Thought:

c:\Ambarish\AZURE-AI-VECTOR-SEARCH\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


The relevant table for the query is "Player". I should now query the schema of the "Player" table to see the available columns.
Action: sql_db_schema
Action Input: Player
Observation: 
CREATE TABLE "Player" (
	id INTEGER, 
	player_api_id INTEGER, 
	player_name TEXT, 
	player_fifa_api_id INTEGER, 
	birthday TEXT, 
	height INTEGER, 
	weight INTEGER, 
	PRIMARY KEY (id)
)

/*
3 rows from Player table:
id	player_api_id	player_name	player_fifa_api_id	birthday	height	weight
1	505942	Aaron Appindangoye	218353	1992-02-29 00:00:00	182.88	187
2	155782	Aaron Cresswell	189615	1989-12-15 00:00:00	170.18	146
3	162549	Aaron Doran	186170	1991-05-13 00:00:00	170.18	163
*/
Thought:

c:\Ambarish\AZURE-AI-VECTOR-SEARCH\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


The "Player" table has the columns id, player_api_id, player_name, player_fifa_api_id, birthday, height, and weight. I can use the "height" column to calculate the average height of the players.
Action: sql_db_query
Action Input: SELECT AVG(height) FROM Player
Observation: [(181.86744484628662,)]
Thought:

c:\Ambarish\AZURE-AI-VECTOR-SEARCH\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


The average height of the players in the database is 181.87.

Final Answer: 181.87

> Finished chain.
{"result": "181.87"}
Invoking: `add_two_numbers` with `{'num1': 181.87, 'num2': 100}`


{"result": 281.87}The average height of players in European Soccer is 181.87. Adding 100 to it gives us a result of 281.87.

> Finished chain.


In [19]:
response["output"]

'The average height of players in European Soccer is 181.87. Adding 100 to it gives us a result of 281.87.'